## AlexNet
Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). ImageNet classification with deep convolutional neural networks. [link]()

### โครงสร้างหลักของ AlexNet

1. Conv

- Filter: 96

- Kernel: 11×11

- Stride: 4

- Padding: 2

- Output: 55×55×96

- Activation: ReLU

- Normalization: LRN

- MaxPooling: 3×3, stride 2

2. Conv

- Filter: 256

- Kernel: 5×5

- Padding: 2

- Stride: 1

- Activation: ReLU

- Normalization: LRN

- MaxPooling: 3×3, stride 2

3. Conv

- Filter: 384

- Kernel: 3×3

- Padding: 1

- Activation: ReLU

4. Conv

- Filter: 384

- Kernel: 3×3

- Padding: 1

- Activation: ReLU

5. Conv

- Filter: 256

- Kernel: 3×3

- Padding: 1

- Activation: ReLU

- MaxPooling: 3×3, stride 2

- Flatten + Fully Connected Layers

6. FC: 4096 neurons → ReLU → Dropout(0.5)

7. FC: 4096 neurons → ReLU → Dropout(0.5)

8. FC: 1000 neurons (หรือ 100 สำหรับ CIFAR-100) → Softmax